In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [2]:
df = pd.read_csv('Crashes (All).csv')

/var/folders/8q/fckhbb_x441bfx284y2c5d8m0000gn/T/ipykernel_45367/3290951917.py:1: DtypeWarning: Columns (6,7,14,15,17,26,29,30,33,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Crashes (All).csv')


In [3]:
df.head()

,CASE_ID,ACCIDENT_YEAR,PROC_DATE,JURIS,COLLISION_DATE,COLLISION_TIME,OFFICER_ID,REPORTING_DISTRICT,DAY_OF_WEEK,CHP_SHIFT,...,COUNT_MC_KILLED,COUNT_MC_INJURED,PRIMARY_RAMP,SECONDARY_RAMP,LATITUDE,LONGITUDE,COUNTY,CITY,POINT_X,POINT_Y
0,81380291,2021,2021-01-04,1942,2021-01-02,1910,35203,2074,6,5,...,0,0,-,-,NaN,NaN,LOS ANGELES,LOS ANGELES,-118.301025,34.043671
1,81380543,2021,2021-01-05,1933,2021-01-01,140,916,30,5,5,...,0,0,-,-,33.941830,-118.339783,LOS ANGELES,INGLEWOOD,-118.338821,33.941811
2,81380545,2021,2021-01-05,1933,2021-01-01,838,828,29,5,5,...,0,0,-,-,33.945438,-118.326469,LOS ANGELES,INGLEWOOD,-118.326477,33.945881
3,81380547,2021,2021-01-05,1933,2021-01-01,1228,828,36,5,5,...,0,0,-,-,33.937698,-118.326508,LOS ANGELES,INGLEWOOD,-118.326721,33.938660
4,81380897,2021,2021-01-05,1942,2021-01-01,920,38282,0638,5,5,...,0,0,-,-,NaN,NaN,LOS ANGELES,LOS ANGELES,-118.309280,34.102161


In [4]:
# Incorporate point x and y into lat long
# Step 1: Fill in missing LAT/LON with POINT_X/POINT_Y
df['LONGITUDE'] = df['LONGITUDE'].fillna(df['POINT_X'])
df['LATITUDE'] = df['LATITUDE'].fillna(df['POINT_Y'])

# Step 2: Select only the useful columns (clean, reduce size)
columns_to_keep = [
    'COLLISION_SEVERITY', 'NUMBER_KILLED', 'NUMBER_INJURED',
    'COUNT_SEVERE_INJ', 'COUNT_VISIBLE_INJ', 'COUNT_COMPLAINT_PAIN',
    'COUNT_PED_KILLED', 'COUNT_PED_INJURED', 'COUNT_BICYCLIST_KILLED', 'COUNT_BICYCLIST_INJURED',
    'COUNT_MC_KILLED', 'COUNT_MC_INJURED', 'PARTY_COUNT',
    'DAY_OF_WEEK', 'COLLISION_TIME', 'CHP_SHIFT',
    'WEATHER_1', 'WEATHER_2', 'LIGHTING', 'ROAD_SURFACE', 'ROAD_COND_1', 'ROAD_COND_2',
    'LOCATION_TYPE', 'INTERSECTION', 'TYPE_OF_COLLISION',
    'PRIMARY_COLL_FACTOR', 'PCF_VIOL_CATEGORY', 'PCF_VIOLATION',
    'HIT_AND_RUN', 'ALCOHOL_INVOLVED',
    'MOTORCYCLE_ACCIDENT', 'TRUCK_ACCIDENT', 'BICYCLE_ACCIDENT', 'PEDESTRIAN_ACCIDENT',
    'LATITUDE', 'LONGITUDE'
]
df = df[columns_to_keep]

# Step 3: Drop rows missing any critical values (required for analysis)
df = df.dropna(subset=['COLLISION_SEVERITY', 'LATITUDE', 'LONGITUDE'])

# Step 4: Convert numeric columns properly
numeric_columns = df.select_dtypes(include=[np.number]).columns
for col in numeric_columns: 
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Step 5: Create binary outcome variable for severity
df['IS_SEVERE'] = df['COLLISION_SEVERITY'].apply(lambda x: 1 if x in [1, 2] else 0)

# Step 6: Drop any completely empty rows, and forward-fill where it makes sense
df = df.dropna(how="all")
df = df.ffill()

# Optional: Check missing value status
nan_count = df.isna().sum()
nan_pct = df.isna().sum() / len(df) * 100
print("Number of missing values per column:\n", nan_count)
print("\nPercentage of missing values:\n", nan_pct.round(2), "%")

# Preview cleaned data
df.head()
#print(df['MOTORCYCLE_ACCIDENT'].value_counts(dropna=False))
# set case ID as index (eisha)

# Connect it to tracts data census (abirami)
# produce contextily map for reference with crs 4326

# produce some basic summary stats

Number of missing values per column:
 COLLISION_SEVERITY            0
NUMBER_KILLED                 0
NUMBER_INJURED                0
COUNT_SEVERE_INJ              0
COUNT_VISIBLE_INJ             0
COUNT_COMPLAINT_PAIN          0
COUNT_PED_KILLED              0
COUNT_PED_INJURED             0
COUNT_BICYCLIST_KILLED        0
COUNT_BICYCLIST_INJURED       0
COUNT_MC_KILLED               0
COUNT_MC_INJURED              0
PARTY_COUNT                   0
DAY_OF_WEEK                   0
COLLISION_TIME                0
CHP_SHIFT                     0
WEATHER_1                     0
WEATHER_2                     0
LIGHTING                      0
ROAD_SURFACE                  0
ROAD_COND_1                   0
ROAD_COND_2                   0
LOCATION_TYPE              3116
INTERSECTION                  0
TYPE_OF_COLLISION             0
PRIMARY_COLL_FACTOR           0
PCF_VIOL_CATEGORY             0
PCF_VIOLATION                 0
HIT_AND_RUN                   0
ALCOHOL_INVOLVED             12
MO

,COLLISION_SEVERITY,NUMBER_KILLED,NUMBER_INJURED,COUNT_SEVERE_INJ,COUNT_VISIBLE_INJ,COUNT_COMPLAINT_PAIN,COUNT_PED_KILLED,COUNT_PED_INJURED,COUNT_BICYCLIST_KILLED,COUNT_BICYCLIST_INJURED,...,PCF_VIOLATION,HIT_AND_RUN,ALCOHOL_INVOLVED,MOTORCYCLE_ACCIDENT,TRUCK_ACCIDENT,BICYCLE_ACCIDENT,PEDESTRIAN_ACCIDENT,LATITUDE,LONGITUDE,IS_SEVERE
0,4,0,2,0,0,2,0,0,0,0,...,21801.0,F,NaN,NaN,NaN,NaN,NaN,34.043671,-118.301025,0
1,3,0,1,0,1,0,0,0,0,0,...,22107.0,N,NaN,NaN,NaN,NaN,NaN,33.941830,-118.339783,0
2,4,0,1,0,0,1,0,0,0,0,...,22350.0,N,NaN,NaN,NaN,NaN,NaN,33.945438,-118.326469,0
3,4,0,1,0,0,1,0,0,0,0,...,21801.0,N,NaN,NaN,NaN,NaN,NaN,33.937698,-118.326508,0
4,3,0,2,0,1,1,0,0,0,0,...,22102.0,N,NaN,NaN,NaN,NaN,NaN,34.102161,-118.309280,0


In [5]:
#ADDING TRACTS W ENVIROSCREEN?
# get the census tract of each collision through joining
enviroscreen = gpd.read_file('../Lectures/data/CalEnviroScreen/CES4 Final Shapefile.shp')
enviroscreen = enviroscreen[enviroscreen.County=='Los Angeles']
df2 = gpd.sjoin(df.to_crs('EPSG:3310'), enviroscreen)

# check there are the same number of rows
print(len(collisionGdf), len(collisionGdf2))

# looks like we lost a few, which we should investigate later on. 
# Maybe there is missing lat/lon? Or spatial imprecision?

# get the tract-level counts
tractcounts = collisionGdf2.groupby('Tract').size()

# we need to give it a name
tractcounts.name = 'n_collisions'

# join back to enviroscreen
enviroscreen = enviroscreen.set_index('Tract').join(tractcounts)

# replace the missing values with zeros (no collisions)
enviroscreen.fillna({'n_collisions': 0}, inplace=True)

# we should do our standard checks here for number of rows, describing the column, a quick-and-dirty plot
collisionGdf2 = gpd.sjoin(collisionGdf.to_crs('EPSG:3310'), enviroscreen)

# check there are the same number of rows
print(len(collisionGdf), len(collisionGdf2))

# looks like we lost a few, which we should investigate later on. 
# Maybe there is missing lat/lon? Or spatial imprecision?

# get the tract-level counts
tractcounts = collisionGdf2.groupby('Tract').size()

# we need to give it a name
tractcounts.name = 'n_collisions'

# join back to enviroscreen
enviroscreen = enviroscreen.set_index('Tract').join(tractcounts)

# replace the missing values with zeros (no collisions)
enviroscreen.fillna({'n_collisions': 0}, inplace=True)

# we should do our standard checks here for number of rows, describing the column, a quick-and-dirty plot

DataSourceError: ../Lectures/data/CalEnviroScreen/CES4 Final Shapefile.shp: No such file or directory

In [ ]:
#CONTEXTILY EXAMPLE

import matplotlib.pyplot as plt
import contextily as ctx

schools.geometry = schools.geometry.centroid
fig, ax = plt.subplots(figsize=(10,10))

# my first try
# looks like we need to make the markers bigger
#schools.plot(markersize='n_collisions', ax=ax)

schools['n_collisions_scaled'] = schools.n_collisions*10
# plot all the schools first, so we don't ignore those ones with zero collisions
schools.to_crs('EPSG:3857').plot(ax=ax, markersize=0.5, color='k')
# plot the proportional markers
schools.to_crs('EPSG:3857').plot(markersize='n_collisions_scaled', color='b', ax=ax)

ctx.add_basemap(ax=ax, alpha=0.5, zoom=13)
ax.set_title('Number of collisions near each school')
ax.set_xticks([])
ax.set_yticks([])

In [ ]:
df['CASE_ID'].is_unique

In [ ]:
df.columns

In [ ]:
# how many sereve/fatal accidents in which tracts, top 10 tracts
# how many severe/datal accidents by weather
# mean number of accidents in top 10 tracts

#boolean values of whether its severe or fatal (eisha)


In [ ]:
# lets go over wrangling and spatial relations

In [ ]:
# 3 maps
# q that just shows crashes in LA
# Machine learning 

In [ ]:


# Step 2: Select only the useful columns (clean, reduce size)
columns_to_keep = [
    'COLLISION_SEVERITY', 'NUMBER_KILLED', 'NUMBER_INJURED',
    'COUNT_SEVERE_INJ', 'COUNT_VISIBLE_INJ', 'COUNT_COMPLAINT_PAIN',
    'COUNT_PED_KILLED', 'COUNT_PED_INJURED', 'COUNT_BICYCLIST_KILLED', 'COUNT_BICYCLIST_INJURED',
    'COUNT_MC_KILLED', 'COUNT_MC_INJURED', 'PARTY_COUNT',
    'DAY_OF_WEEK', 'COLLISION_TIME', 'CHP_SHIFT',
    'WEATHER_1', 'WEATHER_2', 'LIGHTING', 'ROAD_SURFACE', 'ROAD_COND_1', 'ROAD_COND_2',
    'LOCATION_TYPE', 'INTERSECTION', 'TYPE_OF_COLLISION',
    'PRIMARY_COLL_FACTOR', 'PCF_VIOL_CATEGORY', 'PCF_VIOLATION',
    'HIT_AND_RUN', 'ALCOHOL_INVOLVED',
    'MOTORCYCLE_ACCIDENT', 'TRUCK_ACCIDENT', 'BICYCLE_ACCIDENT', 'PEDESTRIAN_ACCIDENT',
    'LATITUDE', 'LONGITUDE'
]